In [16]:
using DataStructures
using Base.Collections
using Iterators
using Pipe
function pz(x :: AbstractArray)
    println(typeof(x), ": ", size(x))
end
macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

In [2]:
using RecursiveAutoencoders

In [3]:
include("load_embeddings.jl")
LL,word_indexes, indexed_words =  load_embeddings("embeddings-test.txt")
#("embeddings-scaled.EMBEDDING_SIZE=50.txt");
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println

(50,14)
Dict{String,Int64}
Array{String,1}


In [4]:
training_trees = open("training_sents.jsz","r") do fs
    deserialize(fs)
end;

In [5]:
abstract Side
immutable Left<:Side
end

immutable Right<:Side
end

immutable NoSide<:Side
end

immutable FoldData
    p::Embedding
    left::Union(FoldData,Embedding)
    right::Union(FoldData,Embedding)
    
end

immutable UnfoldData{T<:Side}
    p::Embedding
    parent::Union(FoldData,UnfoldData)
    ĉ_i::Embedding
    ĉ_j::Embedding
    depth::Int64
end

immutable UnfoldLeaf{T<:Side}
    ĉ::Embedding
    parent::UnfoldData
    c::Embedding
    depth::Int64
end



In [6]:
function fold(rae::RAE, tree::(Any,Any))
    function eval_child(child::String)
        c=eval_word_embedding(rae,child,false)
        c
    end
    function eval_child(c::Embedding)
        c
    end
    function eval_child(child::Any)
        fold(rae,child)
    end
    
    function emb(data::FoldData)
        data.p
    end
    function emb(data::Embedding)
        data
    end
    
    left = eval_child(tree[1])
    right = eval_child(tree[2])
    p=eval_merge(rae, emb(left), emb(right))
    FoldData(p, left, right)   
end

fold (generic function with 1 method)

In [9]:
function unfold{T}(rae::RAE, c::Embedding, ĉ::Embedding, parent, ::Type{T}, depth)
    UnfoldLeaf{T}(ĉ, parent, c, depth)
end


function unfold{T}(rae::RAE, act::FoldData, p::Embedding, parent, ::Type{T}, depth::Int)
    #Side is a ignored argument. This could be replaced with a generated function
    ĉ_i, ĉ_j = reconstruct(rae,p)
    data = UnfoldData{T}(p, parent, ĉ_i, ĉ_j,depth)
    
    left = unfold(rae, act.left, ĉ_i, data, Left, depth+1)
    right= unfold(rae, act.right, ĉ_j, data, Right, depth+1)
    [left; right]
end

function unfold(rae::RAE, act::FoldData)
    #Handle the top case
    unfold(rae, act,act.p,act, NoSide,0)
end

unfold (generic function with 3 methods)

In [12]:
# tests
rae = RAE(LL,word_indexes,indexed_words);

a=fold(rae,("killer", "cows"))
b=unfold(rae,a);
@assert b[1].parent==b[2].parent


a=fold(rae,("the",("killer", "cows")))
b=unfold(rae,a);
@assert b[1].parent==b[2].parent.parent==b[3].parent.parent

In [13]:
function δ(a::Embedding, δ_above::Vector{Float64}, W::Matrix{Float64})
    #a is the ouput of this layer: a=tanh(z) where z is the input from layer below
    #W is matrix to move to above layer, from this one
    dz = 1-a.^2 #Derivitive of a=tanh(z)
    (W'*δ_above).*dz
end

function δ(ĉ_ij::Embedding,c_ij::Embedding) 
    #Output Layer
    M = length(c_ij)# ==length(ĉ_ij)
    dz = 1-ĉ_ij
    δ_above = ĉ_ij-c_ij
    δ(ĉ_ij,δ_above, eye(M))     
end


δ (generic function with 2 methods)

In [14]:
function sidepad(d::Vector{Float64}, ::Left)
    padding=zeros(size(d))
    [padding;d]
end
function sidepad(d::Vector{Float64}, ::Right)
    padding=zeros(size(d))
    [d, padding]
end
function sidepad(d::Vector{Float64}, ::NoSide)
    padding=zeros(size(d))
    [d, padding]
end

function UBPTS{T}(rae::RAE, leaf::UnfoldLeaf{T})
    δ_half = δ(leaf.ĉ,leaf.c)
    sidepad(δ_half, T())
end

function UBPTS{T}(rae::RAE, node::UnfoldData{T}, δ_above::Vector{Float64})
    δ_half = δ([node.p], δ_above, rae.W_d)
    sidepad(δ_half, T())
end

UBPTS (generic function with 2 methods)

In [30]:
xs = DefaultDict(String, Int, 0)
xs["a"]
haskey(xs,"a")

true

In [43]:
methods(enqueue!)

# 1 method for generic function "enqueue!":
enqueue!{K,V}(pq::PriorityQueue{K,V},key,value) at collections.jl:236

In [48]:
function UBPTS(rae::RAE, nodes::Vector{UnfoldLeaf} )
    delta_len = 2*length(nodes[1].ĉ)
    parent_deltas = DefaultDict(UnfoldData, Vector{Float64},zeros(delta_len))
    for node in nodes
        parent_deltas[node.parent]+=UBPTS(rae,node)
    end
    UBPTS(rae,parent_deltas)
end

function UBPTS(rae::RAE, node_deltas::DefaultDict{UnfoldData,Vector{Float64}})
    delta_len = 2*length(first(node_deltas)[1].ĉ_j)
    
    foldnode = nothing
    δ_above_fold = zeros(delta_len)
    
    pending_nodes = PriorityQueue{UnfoldData, Int64}(Base.Order.Reverse)
    enqueue!(node::UnfoldData) = pending_nodes[node] = node.depth
    map(enqueue!, keys(node_deltas)) #Add all that were passed, as none have been processed
    
    function pend!(node::UnfoldData, node_delta::Vector{Float64})
        if !haskey(node_deltas,node)
            enqueue!(node) #then also hasn't been enque
        end
        node_deltas[node]+=node_delta
    end
        
    function pend!(node::FoldData, node_delta::Vector{Float64})
        foldnode = node
        δ_above_fold+=node_delta
    end

    
    while !isempty(pending_nodes)
        node = dequeue!(pending_nodes)
        δ_above =  node_deltas[node]
        pend!(node.parent, UBPTS(rae, node, δ_above))
        @printval typeof(node.parent)
    end
        
    δ_above_fold
end

function UBPTS(rae::RAE, nodes::DefaultDict{Union(UnfoldData,FoldData),Vector{Float64}})
    @printval length(nodes)
    @printval nodes[1]
    @printval nodes[2]
    
    @assert(length(nodes)==1)
    _,δ=first(nodes)
    δ
end



UBPTS (generic function with 5 methods)

In [49]:
UBPTS(rae, b)


typeof(node.parent) = UnfoldData{NoSide}
typeof(node.parent) = FoldData


100-element Array{Float64,1}:
  -8.19354 
  -3.1814  
   2.3397  
  -7.58225 
   1.56845 
  -1.98312 
 -12.9093  
   9.3002  
  -4.7084  
   5.31724 
  -4.30485 
   0.166642
   0.318451
   ⋮       
   0.0     
   0.0     
   0.0     
   0.0     
   0.0     
   0.0     
   0.0     
   0.0     
   0.0     
   0.0     
   0.0     
   0.0     

In [ ]:
function RecursiveAutoencoders.eval_word_embeddings(rae::RAE, tree::(Any,Any))
    function eval_child(child::String)
        eval_word_embedding(rae,child,false)
    end
    function eval_child(child::Any)
        eval_word_embeddings(rae,child)
    end
    c_i = eval_child(tree[1])
    c_j = eval_child(tree[2])
    [c_i c_j]
end

In [ ]:
tree= ("killer", "cows")
c = eval_word_embeddings(rae, tree)
c= [c[:,1], c[:,2]]
p = fold(rae,tree)
ĉ = unfold(rae,tree,p)
ĉ= [ĉ[:,1], ĉ[:,2]]
grad_top(rae, ĉ, c, tree)

In [ ]:
x = Any[2,3]

In [ ]:
x

In [ ]:
function cosine_dist(a,b)
    (a⋅b)/(norm(a)*norm(b))
end

function neighbour_dists(cc::Vector{Float64}, globe::Matrix{Float64})
    [cosine_dist(cc, globe[:,ii]) for ii in 1:size(globe,2)]
end


function show_best(rae::RAE,ĉ::Embedding, nbest=20)
    candidates=neighbour_dists(ĉ,rae.L)   
    best_cands = [ (findfirst(candidates,score), score)
                    for score in select(candidates,1:nbest, rev=true)[1:nbest]]
    vcat([[rae.indexed_words[ii] round(score,2)] for (ii,score) in best_cands]...)
end

function show_bests(rae::RAE,ĉs::Embeddings, nbest=20)
    hcat([show_best(rae,ĉs[:,ii],nbest) for ii in 1:size(ĉs,2)]...)
end


In [ ]:
bs = show_bests(rae, ĉ_ij)
bs[1,:][1:2:end]

In [ ]:
ĉ_ij